# Ch.8 Generative Deep Learning

The potential of artificial intelligence to emulate human thought processes goes beyond passive tasks such as object recognition and mostly reactive tasks such as driving a car. It extends well into creative activities. In the summer of 2015, we were entertained by Google’s DeepDream algorithm turning an image into a psychedelic mess of dog eyes and pareidolic artifacts; in 2016, we used the Prisma application to turn photos into paintings of various styles. In the summer of 2016, an experimental short movie, Sunspring, was directed using a script written by a Long Short-Term Memory (LSTM) algorithm—complete with dialogue. Maybe you’ve recently listened to music that was tentatively generated by a neural network.

Granted, the artistic productions we’ve seen from AI so far have been fairly low quality. AI isn’t anywhere close to rivaling human screenwriters, painters, and composers. But replacing humans was always beside the point: artificial intelligence isn’t about replacing our own intelligence with something else, it’s about bringing into our lives and work more intelligence—intelligence of a different kind. In many fields, but especially in creative ones, AI will be used by humans as a tool to augment their own capabilities: more augmented intelligence than artificial intelligence.

A large part of artistic creation consists of simple pattern recognition and technical skill. And that’s precisely the part of the process that many find less attractive or even dispensable. That’s where AI comes in. Our perceptual modalities, our language, and our artwork all have statistical structure. Learning this structure is what deep-learning algorithms excel at. Machine-learning models can learn the statistical latent space of images, music, and stories, and they can then sample from this space, creating new artworks with characteristics similar to those the model has seen in its training data.

## 8.1 Text generation with LSTM
In this section, we’ll explore how recurrent neural networks can be used to generate sequence data. We’ll use text generation as an example, but the exact same techniques can be generalized to any kind of sequence data: you could apply it to sequences of musical notes in order to generate new music, to timeseries of brushstroke data (for example, recorded while an artist paints on an iPad) to generate paintings stroke by stroke, and so on.

Sequence data generation is in no way limited to artistic content generation. It has been successfully applied to speech synthesis and to dialogue generation for chatbots. The Smart Reply feature that Google released in 2016, capable of automatically generating a selection of quick replies to emails or text messages, is powered by similar techniques.

## 8.1.1 A brief history of generative recurrent networks
In late 2014, few people had ever seen the initials LSTM, even in the machine-learning community. Successful applications of sequence data generation with recurrent networks only began to appear in the mainstream in 2016.
Since then, recurrent neural networks have been successfully used for music generation, dialogue generation, image generation, speech synthesis, and molecule design. They were even used to produce a movie script that was then cast with live actors.

## 8.1.2 How do you generate sequences?
The universal way to generate sequence data in deep learning is to train a network (usually an RNN or a convnet) to predict the next token or next few tokens in a sequence, using the previous tokens as input. For instance, given the input “the cat is on the ma,” the network is trained to predict the target t, the next character. As usual when working with text data, tokens are typically words or characters, and any network that can model the probability of the next token given the previous ones is called a language model. A language model captures the latent space of language: its statistical structure.

Once you have such a trained language model, you can sample from it (generate new sequences): you feed it an initial string of text (called conditioning data), ask it to generate the next character or the next word (you can even generate several tokens at once), add the generated output back to the input data, and repeat the process many times (see image below). This loop allows you to generate sequences of arbitrary length that reflect the structure of the data on which the model was trained: sequences that look almost like human-written sentences. In the example we present in this section, you’ll take a LSTM layer, feed it strings of N characters extracted from a text corpus, and train it to predict character N + 1. The output of the model will be a softmax over all possible characters: a probability distribution for the next character. This LSTM is called a character-level neural language model.

![text gen](images/8_1_2_textgen.jpg)

## 8.1.3 The importance of the sampling strategy
When generating text, the way you choose the next character is crucially important. A naive approach is greedy sampling, consisting of always choosing the most likely next character. But such an approach results in repetitive, predictable strings that don’t look like coherent language. A more interesting approach makes slightly more surprising choices: it introduces randomness in the sampling process, by sampling from the probability distribution for the next character. This is called stochastic sampling.

Sampling probabilistically from the softmax output of the model is neat: it allows even unlikely characters to be sampled some of the time, generating more interesting-looking sentences and sometimes showing creativity by coming up with new, realistic-sounding words that didn’t occur in the training data. But there’s one issue with this strategy: it doesn’t offer a way to control the amount of randomness in the sampling process.

Why would you want more or less randomness? Consider an extreme case: pure random sampling, where you draw the next character from a uniform probability distribution, and every character is equally likely. This scheme has maximum randomness; in other words, this probability distribution has maximum entropy. Naturally, it won’t produce anything interesting. At the other extreme, greedy sampling doesn’t produce anything interesting, either, and has no randomness: the corresponding probability distribution has minimum entropy. **Sampling from the “real” probability distribution—the distribution that is output by the model’s softmax function—constitutes an intermediate point between these two extremes.** But there are many other intermediate points of higher or lower entropy that you may want to explore. Less entropy will give the generated sequences a more predictable structure, whereas more entropy will result in more surprising and creative sequences. When sampling from generative models, it’s always good to explore different amounts of randomness in the generation process. Because we—humans—are the ultimate judges of how interesting the generated data is, interestingness is highly subjective, and there’s no telling in advance where the point of optimal entropy lies.

In order to control the amount of stochasticity in the sampling process, we’ll introduce a parameter called the softmax temperature that characterizes the entropy of the probability distribution used for sampling: it characterizes how surprising or predictable the choice of the next character will be. Let's see the code:

In [2]:
import numpy as np

def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

Higher temperatures result in sampling distributions of higher entropy that will generate more surprising and unstructured generated data, whereas a lower temperature will result in less randomness and much more predictable generated data (see image below).

![reweights](images/8_1_3_reweights.jpg)

## 8.1.4 Implementing character-level LSTM text generation
Let’s put these ideas into practice in a Keras implementation. The first thing you need is a lot of text data that you can use to learn a language model. You can use any sufficiently large text file or set of text files—Wikipedia, The Lord of the Rings, and so on. In this example, you’ll use some of the writings of Nietzsche, the late-nineteenth century German philosopher (translated into English).

**PREPARING THE DATA**<br>
Let’s start by downloading the corpus and converting it to lowercase.

In [4]:
import keras
import numpy as np

path = keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print "Corpus length: ", len(text)

507904/600901 [========================>.....] - ETA: 0sCorpus length:  600901


Next, we’ll extract partially overlapping sequences of length `maxlen`, one-hot encode them, and pack them in a 3D Numpy array x of shape `(sequences, maxlen, unique_characters)`. Simultaneously, we’ll prepare an array `y` containing the corresponding targets: the one-hot-encoded characters that come after each extracted sequence.

**VECTORIZING SEQUENCES OF CHARACTERS**

In [6]:
maxlen = 60 # extract sequences of 60 characters

step = 3 # sample new sequence every 3 characters

sentences = [] #holds the extracted sequences

next_chars = [] # holds the targets (follow-up characters)

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print "Number of sequences: ", len(sentences)

Number of sequences:  200281


In [7]:
chars = sorted(list(set(text))) # list of unique characters in corpus
print 'Unique characters: ', len(chars)
char_indices = dict((char, chars.index(char)) for char in chars) #dict maps unique chars to their index

# One-hot-encode the characters into binary arrays
print "Vectorization..."
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Unique characters:  59
Vectorization...


**BUILDING THE NETWORK**
This network is a single LSTM layer followed by a Dense classifier and softmax over all possible characters. But note that recurrent neural networks aren’t the only way to do sequence data generation; 1D convnets also have proven extremely successful at this task in recent times.

In [8]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

**MODEL COMPILATION CONFIGURATION**<br>
Because our targets are one-hot encoded, we'll use `categorical_crossentropy` as the loss to train the model.

In [9]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

**TRAINING THE LANGUAGE MODEL AND SAMPLING FROM IT**<br>
Given a trained model and a seed text snippet, we can generate new text by doing the following repeatedly:

 1. Draw from the model a probability distribution for the next character, given the generated text available so far.
 2. Reweight the distribution to a certain temperature.
 3. Sample the next character at random according to the reweighted distribution.
 4. Add the new character at the end of the available text.

This is the code we use to reweight the original probability distribution coming out of the model and draw a character index from it (the sampling function).

In [10]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Finally, the following loop repeatedly trains and generates text. You begin generating text using a range of different temperatures after every epoch. This allows you to see how the generated text evolves as the model begins to converge, as well as the impact of temperature in the sampling strategy.

**TEXT-GENERATION LOOP**

In [11]:
import random
import sys

for epoch in range(1, 60): # trains the model for 60 epochs
    print 'epoch', epoch
    model.fit(x, y, batch_size=128, epochs=1) # fits the model for one iteration on the data
    start_index = random.randint(0, len(text) - maxlen - 1) # selects text seed at random
    generated_text = text[start_index: start_index + maxlen]
    print '--- Generating with seed: "' + generated_text + '"'
    
    for temperature in [0.2, 0.5, 1.0, 1.2]: # tries a range of different sampling temperatures
        print '------ temperature:', temperature
        sys.stdout.write(generated_text)

        for i in range(400): # generates 400 characters, starting from the seed text
            sampled = np.zeros((1, maxlen, len(chars))) # one-hot encodes the chars generated so far
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0] # samples the next character
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)

epoch 1
Epoch 1/1
200281/200281 [==============================] - 953s - loss: 1.9907   
--- Generating with seed: "e result of defective
perception, and this from absolute nec"
------ temperature: 0.2
e result of defective
perception, and this from absolute necessity of the sach and still to the sense of the santing to the senses to the sense and still to the sense and sense of the sign to the senses of the self that is the fact and still and so such the conceition of the still and so that the same to the self to the sense and sense and so have the sense and sensing to the sense and so have to the such the sense for the still and still and still and suc------ temperature: 0.5
the such the sense for the still and still and still and such is love and at and the senseity and has like and things to has the socture of no lore and the such frening to the mastent, and so indigination of conscious voluse and prover that of the prower in the rast that the prode mis have condition of that the 

al live stifled be redains of them. ethic.--accient sial yeter, expresio, self-lligy he lone wisling to ourges applanis will be it hanchine--or resden abought the fevenive-pares the rahess of their eecy, from this most moundouriw fencaraueaze, almoticalut
lose very wirdums pilitieepoch 5
Epoch 1/1
200281/200281 [==============================] - 961s - loss: 1.4708   
--- Generating with seed: "wes itself.

261. vanity is one of the things which are perh"
------ temperature: 0.2
wes itself.

261. vanity is one of the things which are perhaps the same stand the instincts of the same to stand the same the same that the suffering to the same stand the same to be stand the same things to the superion of the belief to the superston the same of the superions of the superion of the superion of the stand in the superions of the has a supersting and the same things is the same to the same to the superion of the compation of the persis and ------ temperature: 0.5
the same to the superion of the 

appage do not timaly and some who plaacvels of appeared in the mosts when hammom, learnt to become were and mechomaute,
somenkton simpliz------ temperature: 1.2
mom, learnt to become were and mechomaute,
somenkton simplizy" the most mears with necessaritudes itself and slacsia, emflestio alvoy: not tantage, is not recunt groalfweingest-found,
imomstany the nulvusate, grancefqence as not exceprion
of the pies.--aw externate wao is higher then, man in its kd"; with a heffens men priciser's inhold percey i
diyes it isunvon. longed means, it fapt imman expectian
musses of realt ammiriashy, non being opposel byecnes.reepoch 9
Epoch 1/1
200281/200281 [==============================] - 892s - loss: 1.4073   
--- Generating with seed: "rejoice on account of praise is in many cases merely politen"
------ temperature: 0.2
rejoice on account of praise is in many cases merely politeness of the command of the commanding there is a still there is also been the personal there is a personal there are 

ction of the certain our suntrament, more heart of musicism of
the
decold, in the inquirement in knowledged are the metately ramptable strictive of a lect itself, images only extensive your openes the takes without we stor moral, the pressing him tared predicious intoterable storn! a favour: and a
virtu far also same and letscy smor. the relic. they
are
refief more more more have a surerition, has
still hostification,
and it is lack as its love not of unde------ temperature: 1.2

still hostification,
and it is lack as its love not of understanciations
liftic "dopher,
o ; but"y too spine, camilicors forest, was pleasure no wefent. follti, living of gradude fooly of the savaless"--cast caoly
si ideede, "my "lifting decageium and experience, disposition. that her thorouss or moralty nor
in the
french onces
soint indeed, certimed ridach a clisonce,
gractiony,
books; could generaceness: german upon a cons -more educal denyed, instrictiepoch 13
Epoch 1/1
200281/200281 [======================

/Volumes/RobStorage/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


...

�f��wden�xv�ch �s��goera6�l�00 nae  e vo�d9� 3o�ryp0lenre p�t eniu  0�o��q��0d �the 0�j0t t�c�bt�thete e0� sh� �f te    0es[ 0d fa e h�q�dne�y��� g�����t��k �w�0lardeone   ���q--ragen the e6u��t o �jen    ���qu�el�e��sfu��wa�th����t�ju���b���t�� roo  0s      �     9ea  we ea�  of   �te  e ��kps � �qla�k t��cuepoch 44
Epoch 1/1
200281/200281 [==============================] - 816s - loss: 16.1133   
--- Generating with seed: "ts no demonstrable supremacy and a struggle leads but to
mut"
------ temperature: 0.2
ts no demonstrable supremacy and a struggle leads but to
mut�069��609�0�wath(0�-6s���ly0e�qt��qu6q6 �q9�w�j presf  on 0eh�96. at��e 6whehe�0he thh�9�'6sher � 6a�d�0 t�qs�ju ate an� 5.     no�    �(z�fd she 0o� �0zgh-ees h��t h0�j. th�ng� the ����wh�(wh�0t at�jst the  o�6�qe0. �whhe6�q-��"ba�d�w�j�6 the 9 [
���k�jky���pt �pppe or  �   �ne ohe  xhe�6����0 the�0m�j� ean ���b6��wg��5 nohy0th0y0.     the � �hgend �j0 0o�ofes�(ud 0.��6�j69�q��q6t�jg��6�6wtde s�j------ temperature: 0.5
h

 e    e e e     e e e e e e  e e e e e e e e     e e e ene    e e e e     e4e  e    e e pe e e e  e e e le   e e e e e    e e e    e ee e e e e e e e e e e      e e e e    e e e      e  e          e e e e e       e     e  e e e e e e e e e e e e e  e e e e     e e e           e e e [ e e      e e e e e e      e      ee e    e e e e e      e e e e e e e e e    e e e  e e e e e e e e e e e e e e    e        e e e    e e e       e e      e e e e e e me     e ------ temperature: 1.2
e        e e e    e e e       e e      e e e e e e me     e e e e  e e        e e    e e    e ee e e e e e e e e e e 4 e e e e e   e e e e e e e e      e    e     e e    e e e   e e e    e             e e e e e e e e e      ee e e e e e e e e   e e e  e      e e  e e e [ e e e e e  e       e e fe e      e e  e e      e e          e e    7     e e e e e e  e      e  e e  e e e           e e e  4e   " e  e e e e e e e e e  e      e e e e e     e e e e e e   epoch 48
Epoch 1/1
200281/200281 [======================

            e   7          he     [              [ e      pn                        ne e e e  e     4e          e         ee           6e   p  e      e e   e ee          n  ee            e  e       e e  e       e  e    e  e    ee  e    8   e    ee      e e e  4e  e e      ee e           e          e  e     7              ee     ee ee     e e     3      e          e        e   n e   7    e  ee         e            e e e e e     e e  e ee e e   e  d         ------ temperature: 1.0
    e            e e e e e     e e  e ee e e   e  d             e e  e    e  ee  e  e   e     e        e ee  e     e e e e  e  e  e  e  e    e e e    e ee    e      ee e           e  e 9              e e  e       e  e e       e  ee e       ee         e f ee  e     e ee        e    e e e ee    e  ee         e    e e    ee               e  e                 e      ne     ee    e  me       e ee     e         e  e   e e e    e  e       e    e     e    e ee e e------ temperature: 1.2
         e  e   e e e    e  e   

regard to the beloved. "we e  ee e e e e ee ee e e ee e e e ee  e e e  e eee  e e e eee e  e eee ee e e ee e   e ee e ee e e e e e  e e  e e ee ee e   e ee e e ee  e e e eee e e  e e e e ee e e e e e ee e  e eee  ee e e e ee e ee e e e  e ee eee e e e e ee e ee e ee e  e e ee e  ee e e e e eee e e e  ee e  ee e e e  e e  e  e ee   e e ee e e e e e ee e e e e e e ee e  ee e   e ee e e e e ee ee eee e e  e ee e e e ee e ee ------ temperature: 0.5
ee e  ee e   e ee e e e e ee ee eee e e  e ee e e e ee e ee    ee e e e e e e e e  e e e e ee ee e e ee ee e ee e e eee e ee e e e e e e ee   e  ee e ee  e e e ee      e ee e   eee e ee e  e ee  e e e ee ee e  e ee e ee e e e  e e ee e ee ee e  e e e ee   e e e e ee e ee e e e e e ee e e  e e e ee e e ee e ee e e e ee  ee ee e e  ee ee e e eee e e e e e ee e e e   e e  eee e ee  eee ee e eee ee   ee e eeee eeee e  e e e ee e e     e e e e ee e ee   e e e e------ temperature: 1.0
ee e eeee eeee e  e e e ee e e     e e e e ee e ee   e e e ee ee e 

e e eeee e  e ee  e eetee ee e e     e  e e  e e  e e ee e e   e e e qe  eeeee  e e ee ee e ee ee e ee ee eeee     e e ee eee e e e  e   e ee e  eee e e   e ee  ee  ee e e e ee e e  e e ee ee  e  e e eee ze e ee   eee ee e ee ee e e    he ee ee ee ee    e  e   e eee ee  eee  e e e ee e e e   e e   ee e e    ee ee e e ee    eee e eeee eee ee e e  e  e e e e e e e  e e ee   e  ee ee e e e ee e  e e e"   eeee    e e ee ee   e eeeeee e  e e ee eee     ee  ee eepoch 59
Epoch 1/1
200281/200281 [==============================] - 796s - loss: 12.2228   
--- Generating with seed: "ciences,
for whose service and equipment the other sciences "
------ temperature: 0.2
ciences,
for whose service and equipment the other sciences      e  e      e   ee e    e     ee    e                  ze    e efq
   e exhe   he e   e d      ee e e e   e e  e n  e e ee  e eo   e e e e  e  e e  e e ee e       e  n e  e     e  e e   w he
e e  e   s e e  n we  e1e   e e  e   x  e    e  eed e e 'e e e  e ee e  n e    he

Here, we used the random see text "stly band, who involuntarily communicate their own tension t." Here's what we get at epoch 20 when we set the temperature to 0.2:

`stly band, who involuntarily communicate their own tension that it is the sentiments of the sentiments and the contradiction of the senter standard to the desired and in the result of the religious and the strength. the sering to the sense of the most distinction of the saint of the same taste of the character of the proves the end the standards to the fact that the standard of the standard of an antitus the standard of the man is all the basis of the same`

Here it is with `temperature = 0.5`:

`the spirit of the fact in the same subtle the sense of the same saint that the spirit of his origin to possess of order to a germans, the self-univels of the contectlies of the spirit and the finer some tradition of in the best other is the artious to from the defictions of mankind, and the best feelings of the ferropation of the popular and thoughts and happy profound, the intenses and its origin of the craved and stronger and self-men, in the last power`

And `temperature = 1.0`:

`of the craved and stronger and self-men, in the last power nothing. they strength tho oving ocloge, a present confinience over--to the greatehermy need, everyrorrmben; of the inst enjoy. there foug? therein, and thing immutal, of the armivation to which not in its to geniuss of a man that rulely allessess they wanter an additate, is nok he will: weraves, are new self-work youth, weaked upon their higher of slowly equally beact in less i clealicy full-such`

As we can see, a low temperature value results in extremely repetitive and predictable text, but local structure is highly realistic: in particular, most words are real English words. With higher temperatures, the generated text becomes more interesting, surprising, even creative; it sometimes invents completely new words that sound like they could be considered in a Scrabble game.

By training a bigger model, longer, on more data, we can achieve generated samples that look much more coherent and realistic than this one.

## 8.1.5 Wrapping up
 - You can generate discrete sequence data by training a model to predict the next token(s), given previous tokens.
 - In the case of text, such a model is called a language model. It can be based on either words or characters.
 - Sampling the next token requires balance between adhering to what the model judges likely, and introducing randomness.
 - One way to handle this is the notion of softmax temperature. Always experiment with different temperatures to find the right one.